In [1]:
library("AnnotationDbi")
library("org.Hs.eg.db")
library("enrichplot")
library("clusterProfiler")


Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Loading required package: IRanges

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The f

In [2]:
# packageVersion("igraph")


> ## Calcitriol Stimulation

In [3]:
## Load DE genes from Vit-D stimulated TH2 cells
res <- read.csv('Supplementary/Tab_S4.csv', row.names = 1 ) 

In [4]:
## https://learn.gencore.bio.nyu.edu/rna-seq-analysis/gene-set-enrichment-analysis/
de_genes_list <-  res #rbind(fcvals_down, fcvals_up)
de_genes_list  <- de_genes_list#[de_genes_list$gene_biotype == 'protein_coding',]
# de_genes_list <- de_genes_list[abs(de_genes_list$logFC) >= 0.58, ]

# Create a vector of the gene unuiverse
gene_list <- de_genes_list$logFC

# Name vector with ENTREZ ids
names(gene_list) <- de_genes_list$entrezgene_id #annot[rownames(de_genes_list),]$entrezgene_id

# omit any NA values 
gene_list <- na.omit(gene_list)

# sort the list in decreasing order (required for clusterProfiler)
gene_list = sort(gene_list, decreasing = TRUE)

gene <- names(gene_list)
geneList <- gene_list


go <- enrichKEGG(gene         = gene,
                organism     = 'mmu',
                pvalueCutoff = 0.01,
                # nPerm        = 10000,
                minGSSize    = 10,
                maxGSSize    = 2000,
                pAdjustMethod = "BH")
dim(go)

go <- as.data.frame(setReadable(go, 'org.Mm.eg.db',keyType="ENTREZID"))
# go$Description
# as.data.frame(go)


li_gr = c()
li_br = c()
for (x in go$GeneRatio) {
  li_gr <- append(li_gr, (eval(parse(text=x))) )
}

for (x in go$BgRatio) {
  li_br <- append(li_br, (eval(parse(text=x))) )
}
tmp <- data.frame(go)
tmp$FE <- li_gr/li_br ## Fold enrichment
tmp$log10adjpval <- -log10(tmp$p.adjust)
tmp$rankFDR <- c(seq(1, nrow(tmp)))
tmp <- tmp[order( -tmp$FE),]
tmp$rankFE <- seq(1, nrow(tmp))
tmp$rank <- (tmp$rankFDR + tmp$rankFE)/2
# tmp <- tmp[order(-tmp$log10adjpval),]


tab <- tmp[order(-tmp['log10adjpval']),]
# tab <- tab[1:10,]
tab <- tab[c('ID','Description','FE','log10adjpval','Count')]
tab$nGenes <- tab$Count
tab['-Log10FDR'] <- tab$log10adjpval
tab <- tab[order(tab$FE),]
tab$Description <- gsub("\\- Mus musculus \\(house mouse\\)", "", tab$Description)
# tab$Pathway <- gsub("\\- Mus musculus \\(house mouse\\)", "", tab$Pathway)
tab$Pathway <- factor(tab$Description, levels = tab$Description)


tab

Reading KEGG annotation online: "https://rest.kegg.jp/link/mmu/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/mmu"...



[1] 15  9



Warning message in xtfrm.data.frame(x):
“cannot xtfrm data frames”


,ID,Description,FE,log10adjpval,Count,nGenes,-Log10FDR,Pathway
,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>,<fct>
mmu04010,mmu04010,MAPK signaling pathway,2.044153,2.069134,26,26,2.069134,MAPK signaling pathway
mmu05417,mmu05417,Lipid and atherosclerosis,2.300764,2.130798,21,21,2.130798,Lipid and atherosclerosis
mmu05202,mmu05202,Transcriptional misregulation in cancer,2.429888,2.613116,23,23,2.613116,Transcriptional misregulation in cancer
mmu04148,mmu04148,Efferocytosis,2.645776,2.400505,18,18,2.400505,Efferocytosis
mmu04060,mmu04060,Cytokine-cytokine receptor interaction,2.736769,4.916766,34,34,4.916766,Cytokine-cytokine receptor interaction
mmu05146,mmu05146,Amoebiasis,3.096355,2.400505,14,14,2.400505,Amoebiasis
mmu04659,mmu04659,Th17 cell differentiation,3.155333,2.410379,14,14,2.410379,Th17 cell differentiation
mmu04210,mmu04210,Apoptosis,3.306140,3.505959,19,19,3.505959,Apoptosis
mmu04061,mmu04061,Viral protein interaction with cytokine and cytokine receptor,3.487474,2.673657,14,14,2.673657,Viral protein interaction with cytokine and cytokine receptor


In [5]:
size =12 

library(ggplot2) 

pdf("Supplementary/Fig_S3.pdf", width = 9, #7.5,
    height = 4)
# Horizontal version
p <- ggplot(data = tab, aes(x = Pathway, y = FE ) ) + 
# scale_color_gradient(low = "#a6b4a6", high = "#404e40") +
scale_color_gradient(low = "#e0bd8e", high = "#8e662f") +
  geom_segment(  aes(x=Pathway, xend=Pathway, y=0, yend=FE, color = `-Log10FDR`, size = 1), show.legend = F ) +
  geom_point(data = tab, aes(color = `-Log10FDR`, size = nGenes)) + 
  theme_light() +
  coord_flip() +
  xlab("PATHWAYS") + 
  ylab("Fold Enrichment") + 
  ggtitle("KEGG ANALYSIS")+
  theme(
    panel.grid.major.y = element_blank(),
    axis.ticks.y = element_blank(),
    axis.text.x = element_text(size=15, angle=0),
    axis.text.y = element_text(size=15, angle=0),
    plot.title = element_text(size=12, face='bold', hjust = 0.5),
    panel.background = element_blank(),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    panel.border = element_rect(colour = "grey", fill=NA, size=1),
    axis.title.y = element_text(size = size),
    axis.title.x = element_text(size = size), 
      legend.title = element_text(size=12),
      legend.text = element_text(size=12)
  )
p
dev.off()

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”
Warning message:
“The `size` argument of `element_rect()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


png 
  2

> ## Vit-D  Deficient

In [6]:
res <- read.csv('Supplementary/Tab_S6.csv', row.names = 1 )
dim(res)


[1] 886  10

In [7]:
## https://learn.gencore.bio.nyu.edu/rna-seq-analysis/gene-set-enrichment-analysis/
library(org.Mm.eg.db)
de_genes_list <-  res
de_genes_list  <- de_genes_list[de_genes_list$gene_biotype == 'protein_coding',]
# de_genes_list <- de_genes_list[abs(de_genes_list$logFC) >= 0.58, ]

# Create a vector of the gene unuiverse
gene_list <- de_genes_list$logFC

# Name vector with ENTREZ ids
names(gene_list) <- de_genes_list$entrezgene_id #annot[rownames(de_genes_list),]$entrezgene_id

# omit any NA values 
gene_list <- na.omit(gene_list)

# sort the list in decreasing order (required for clusterProfiler)
gene_list = sort(gene_list, decreasing = TRUE)
# names(gene_list)
kegg <- enrichKEGG(gene         = names(gene_list),
                 organism     = 'mmu',
                  pvalueCutoff = 0.01,
                 # nPerm        = 10000,
               minGSSize    = 10,
               maxGSSize    = 2000,
               pAdjustMethod = "BH"
                )

go <- as.data.frame(setReadable(kegg, 'org.Mm.eg.db',keyType="ENTREZID"))

li_gr = c()
li_br = c()
for (x in go$GeneRatio) {
  li_gr <- append(li_gr, (eval(parse(text=x))) )
}

for (x in go$BgRatio) {
  li_br <- append(li_br, (eval(parse(text=x))) )
}
tmp <- data.frame(go)
tmp$FE <- li_gr/li_br ## Fold enrichment
tmp$log10adjpval <- -log10(tmp$p.adjust)
tmp$rankFDR <- c(seq(1, nrow(tmp)))
tmp <- tmp[order( -tmp$FE),]
tmp$rankFE <- seq(1, nrow(tmp))
tmp$rank <- (tmp$rankFDR + tmp$rankFE)/2
# tmp <- tmp[order(-tmp$log10adjpval),]


tab <- tmp[order(-tmp['log10adjpval']),]
# tab <- tab[1:10,]
# tab
# tab
tab <- tab[c('ID','Description','FE','log10adjpval','geneID','Count')]
tab$nGenes <- tab$Count
tab['-Log10FDR'] <- tab$log10adjpval
tab <- tab[order(tab$FE),]
tab$Description <- gsub("\\- Mus musculus \\(house mouse\\)", "", tab$Description)
tab$Pathway <- factor(tab$Description, levels = tab$Description)
dim(tab)
tab

Warning message in xtfrm.data.frame(x):
“cannot xtfrm data frames”


[1] 14  9

,ID,Description,FE,log10adjpval,geneID,Count,nGenes,-Log10FDR,Pathway
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<dbl>,<fct>
mmu04060,mmu04060,Cytokine-cytokine receptor interaction,2.423451,2.382618,Il23a/Ccl3/Ccl27a/Ccl4/Il1a/Csf2/Ifng/Ccl17/Cxcr3/Eda2r/Il6/Tnfsf4/Il12rb2/Lta/Il2/Cxcr4/Tnfsf15/Acvr1c/Il18r1/Il1rl1/Tnfrsf19,21,21,2.382618,Cytokine-cytokine receptor interaction
mmu05166,mmu05166,Human T-cell leukemia virus 1 infection,2.609870,2.443466,H2-T3/Csf2/H2-Q7/Il6/H2-K1/Cdkn1a/H2-D1/Bax/Adcy3/Lta/Icam1/Il2/H2-Q2/Map3k1/Fdps/Ccnb2/H2-Ob/Creb5/Gm11127,19,19,2.443466,Human T-cell leukemia virus 1 infection
mmu04145,mmu04145,Phagosome,2.843155,2.208120,H2-T3/H2-Q7/Thbs1/Clec7a/Olr1/H2-K1/Tubb3/H2-D1/H2-Q2/Itgb3/C3/Tuba8/H2-Ob/Gm11127/Atp6v0e2,15,15,2.208120,Phagosome
mmu04061,mmu04061,Viral protein interaction with cytokine and cytokine receptor,3.571402,2.014570,Ccl3/Ccl27a/Ccl4/Ccl17/Cxcr3/Il6/Lta/Il2/Cxcr4/Il18r1,10,10,2.014570,Viral protein interaction with cytokine and cytokine receptor
mmu05416,mmu05416,Viral myocarditis,4.056646,2.603780,H2-T3/H2-Q7/Cd80/H2-K1/Prf1/H2-D1/Icam1/H2-Q2/H2-Ob/Myh7/Gm11127,11,11,2.603780,Viral myocarditis
mmu04650,mmu04650,Natural killer cell mediated cytotoxicity,4.171514,3.702143,Klra7/Klrc1/Csf2/Ifng/Klra4/Klra1/Klrd1/Gzmb/Klrk1/Klra3/H2-K1/Prf1/Icam2/H2-D1/Icam1,15,15,3.702143,Natural killer cell mediated cytotoxicity
mmu04612,mmu04612,Antigen processing and presentation,4.289787,2.779164,H2-T3/Klrc1/Ifng/H2-Q7/Klrd1/H2-K1/Cd8b1/H2-D1/H2-Q2/H2-Ob/Gm11127,11,11,2.779164,Antigen processing and presentation
mmu05323,mmu05323,Rheumatoid arthritis,4.289787,2.779164,Il23a/Ccl3/Il1a/Csf2/Ifng/Il6/Angpt1/Cd80/Icam1/H2-Ob/Atp6v0e2,11,11,2.779164,Rheumatoid arthritis
mmu05320,mmu05320,Autoimmune thyroid disease,4.910677,3.218805,H2-T3/H2-Q7/Gzmb/Cd80/H2-K1/Prf1/H2-D1/Il2/H2-Q2/H2-Ob/Gm11127,11,11,3.218805,Autoimmune thyroid disease


In [8]:
pdf("Figures/Fig5H.pdf", width = 7.5, height = 4)
# Horizontal version
p <- ggplot(data = tab, aes(x = Pathway, y = FE ) ) + 
scale_color_gradient(low = "#bddad7", high = "#567e7b") +
  geom_segment(  aes(x=Pathway, xend=Pathway, y=0, yend=FE, color = `-Log10FDR`, size = 1), show.legend = F ) +
  geom_point(data = tab, aes(color = `-Log10FDR`, size = nGenes)) + 
  theme_light() +
  coord_flip() +
  xlab("PATHWAYS") + 
  ylab("Fold Enrichment") + 
  ggtitle("KEGG ANALYSIS")+
  theme(
    panel.grid.major.y = element_blank(),
    axis.ticks.y = element_blank(),
    axis.text.x = element_text(face="bold", #color="#008000",
                               size=15, angle=0),
    axis.text.y = element_text(face="bold", #color="#008000",
                           size=15, angle=0),
    plot.title = element_text(size=12, face='bold', hjust = 0.5),
    panel.background = element_blank(),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    panel.border = element_rect(colour = "grey", fill=NA, size=1),
    axis.title.y = element_text(size = size),
    axis.title.x = element_text(size = size), 
      legend.title = element_text(size=12),
      legend.text = element_text(size=12)
  )
p
dev.off()


png 
  2

## Graph

In [9]:
library("ggplot2")
library("enrichplot")
edox <- readRDS("Supplementary/edox.rds")

In [ ]:
down <- "#adc0a5"
up <-  "#6dc4bc"

de_genes_list <- read.csv('Supplementary/Tab_S6.csv', row.names = 1 ) #rbind(fcvals_down, fcvals_up)
de_genes_list  <- de_genes_list[de_genes_list$gene_biotype == 'protein_coding',]
# Create a vector of the gene unuiverse
gene_list <- de_genes_list$logFC
# Name vector with ENTREZ ids
names(gene_list) <- de_genes_list$ensembl_gene_id #entrezgene_id #annot[rownames(de_genes_list),]$entrezgene_id
# omit any NA values 
gene_list <- na.omit(gene_list)

pdf(file = "Figures/Fig_5E.pdf",  bg = "transparent", width = 7, height= 5)#, res = 50)   
cnetplot(edox,
        node_label = "all",
        # node_label="gene", 
        showCategory = 5,
        # layout = 'nicely',#'nicely',
        edge.width = 0.00001,
        cex_label_gene = 0.75, 
        color_category='lightslateblue', 
        cex_label_category = 0, #1.5,
        categorySize= "log10.p.adjust" , 
        node_label_size = 10,
        foldChange= gene_list) + 
                scale_color_gradient2(name = "-Log10Pval", low = down, mid = "white",
                                      high = up, midpoint = 0, space = "Lab", guide = "colourbar") +
        ggplot2::theme(legend.text=element_text(size=16, face = "bold"),
                        text = element_text(size = 20, face = "bold"),
                        legend.title=element_text(size=16, face = "bold"),
                        plot.title = element_text(size=20, face = "bold"))


dev.off()
cnetplot(edox)



Warning message in cnetplot.enrichResult(x, ...):
“Use 'color.params = list(foldChange = your_value)' instead of 'foldChange'.
 The foldChange parameter will be removed in the next version.”
Warning message in cnetplot.enrichResult(x, ...):
“Use 'color.params = list(category = your_value)' instead of 'color_category'.
 The color_category parameter will be removed in the next version.”
Warning message in cnetplot.enrichResult(x, ...):
“Use 'cex.params = list(category_label = your_value)' instead of 'cex_label_category'.
 The cex_label_category parameter will be removed in the next version.”
Warning message in cnetplot.enrichResult(x, ...):
“Use 'cex.params = list(gene_label = your_value)' instead of 'cex_label_gene'.
 The cex_label_gene parameter will be removed in the next version.”
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Scale for colour is already present.
Adding another scale for colour, which will replace the existing

png 
  2